In [1]:
import pandas as pd
import json
import os
import numpy as np
import difflib
import re

from tqdm import tqdm
from collections import deque, namedtuple

from IPython.display import display

# Listened log

In [2]:
DATA = './source/music.json'
SCHEMA = 'android'
TRACKS = './source/tracks/'

FIX_ARTISTS = {
    'Tarja Turunen': 'Tarja',
    'Thomas Bergersen': 'Two Steps from Hell',
    'Томас Бергерсен': 'Two Steps from Hell',
    'Two Steps from Hell & Thomas Bergersen': 'Two Steps from Hell',
    'Tribe': 'Amaranthe',
    'Two Steps From Hell': 'Two Steps from Hell',
    'Two Steps From Hell, Thomas Bergersen': 'Two Steps from Hell',
    'Two Steps From Hell, Nick Phoenix': 'Two Steps from Hell',
    'Андреас Вальдетофт': 'Andreas Waldetoft',
    'Andreas Waltedoft': 'Andreas Waldetoft',
    'Paradox Interactive': 'Andreas Waldetoft',
    'Meyer': 'Andreas Waldetoft'
}

FIX_TITLES = {
    'inf': 'Infinity'
}

def fix_artist(artist):
    return FIX_ARTISTS.get(artist, artist)

def fix_title(title):
    title = re.sub(r" ?\([^)]+\)", "", title)
    title = re.sub(r"Listened to", "", title)
    title = title.strip()
    return FIX_TITLES.get(title, title)

def check_title(title):
    return "Skipped" not in title and "Google Play Music" not in title

with open(DATA, 'r') as f:
    data = json.load(f)

In [3]:
df = pd.DataFrame(data)
df

,header,title,description,time,products,subtitles
0,Google Play Music,Listened to Resign to Surrender ~ A New Age Da...,Epica,2020-06-24T12:33:36.733Z,[Google Play Music],NaN
1,Google Play Music,Listened to Samadhi - Prelude,Epica,2020-06-24T12:27:17.606Z,[Google Play Music],NaN
2,Google Play Music,Listened to Céilí,Xandria,2020-06-23T16:24:38.922Z,[Google Play Music],NaN
3,Google Play Music,Listened to Ship of Doom,Xandria,2020-06-23T16:21:17.117Z,[Google Play Music],NaN
4,Google Play Music,Listened to When the Walls Came Down (Heartach...,Xandria,2020-06-23T16:16:26.869Z,[Google Play Music],NaN
...,...,...,...,...,...,...
12639,Google Play Music,Listened to Ocean Princess,Томас Бергерсен,2019-05-12T09:20:50.609Z,[Google Play Music],NaN
12640,Google Play Music,Listened to Hurt,Томас Бергерсен,2019-05-12T09:17:57.848Z,[Google Play Music],NaN
12641,Google Play Music,Listened to Dreammaker,Томас Бергерсен,2019-05-12T09:16:14.505Z,[Google Play Music],NaN
12642,Google Play Music,Listened to Starvation,Томас Бергерсен,2019-05-12T09:11:56.597Z,[Google Play Music],NaN


In [4]:
df['artist'] = df['description'].apply(fix_artist)
df['title'] = df['title'].apply(fix_title)
df = df[[check_title(t) for t in df.title]]
df = df.reset_index(drop=True)
df = df[['artist', 'title', 'time']]
df

,artist,title,time
0,Epica,Resign to Surrender ~ A New Age Dawns - prt IV ~,2020-06-24T12:33:36.733Z
1,Epica,Samadhi - Prelude,2020-06-24T12:27:17.606Z
2,Xandria,Céilí,2020-06-23T16:24:38.922Z
3,Xandria,Ship of Doom,2020-06-23T16:21:17.117Z
4,Xandria,When the Walls Came Down,2020-06-23T16:16:26.869Z
...,...,...,...
12407,Two Steps from Hell,Ocean Princess,2019-05-12T09:20:50.609Z
12408,Two Steps from Hell,Hurt,2019-05-12T09:17:57.848Z
12409,Two Steps from Hell,Dreammaker,2019-05-12T09:16:14.505Z
12410,Two Steps from Hell,Starvation,2019-05-12T09:11:56.597Z


# Tracks data

In [5]:
files = [f for f in os.listdir(TRACKS) if os.path.isfile(os.path.join(TRACKS, f))]

tracks = []

for file in tqdm(files):
    df_f = pd.read_csv(os.path.join(TRACKS, file))
    obj = df_f.to_dict(orient='index')[0]
    tracks.append(obj)
    
df_t = pd.DataFrame(tracks)
df_t.head()

100%|██████████| 1383/1383 [00:03<00:00, 404.51it/s]


,Title,Album,Artist,Duration (ms),Rating,Play Count,Removed
0,Monopoly on Truth,Requiem for the Indifferent,Epica,431359,0,7,NaN
1,Tragica (&#39;tragedienne&#39; Spanish Version...,The Seventh Life Path,Sirenia,294870,0,0,NaN
2,Why not me,The Unforgiving,Within Temptation,34560,0,1,NaN
3,Mercy in Darkness,Archangel,Two Steps from Hell,70974,0,6,NaN
4,Molto Piratissimo,Unleashed,Two Steps from Hell,188499,0,4,NaN


In [6]:
import html
df_t['Title'] = df_t['Title'].apply(lambda t: html.unescape(str(t))).apply(fix_title)
df_t['Album'] = df_t['Album'].apply(lambda t: html.unescape(str(t)))
df_t['Artist'] = df_t['Artist'].apply(lambda t: html.unescape(str(t))).apply(fix_artist)
df_t.columns = 'title', 'album', 'artist', 'duration', 'rating', 'play_count', 'removed'
df_t = df_t.drop(['rating', 'removed'], axis=1)
df_t.head()

,title,album,artist,duration,play_count
0,Monopoly on Truth,Requiem for the Indifferent,Epica,431359,7
1,Tragica,The Seventh Life Path,Sirenia,294870,0
2,Why not me,The Unforgiving,Within Temptation,34560,1
3,Mercy in Darkness,Archangel,Two Steps from Hell,70974,6
4,Molto Piratissimo,Unleashed,Two Steps from Hell,188499,4


# Join listened with tracks

In [7]:
tracks_by_name = {}

for track in df_t.itertuples(index=True):
    try:
        tracks_by_name[track.title].append(track)
    except KeyError:
        tracks_by_name[track.title] = [track]
        
track_names = list(tracks_by_name.keys())

In [8]:
def get_tracks(title, cutoff=0.6):
    try:
        track = tracks_by_name[title]
        return track
    except KeyError:
        pass
    if len(title) > 5:
        try:
            matches = [t for t in track_names if title in t]
            if len(matches) > 2:
                match = difflib.get_close_mathces(title, matches, cutoff=0.4)[0]
            else:
                match = matches[0]
            return tracks_by_name[match]
        except (StopIteration, IndexError):
            pass
    elif "Instrumental" in title:
        title = re.sub('Instrumental', "", title)
        title = re.sub('-', "", title)
        title = title.strip()
        return get_tracks(title, cutoff)
    matches = difflib.get_close_matches(title, track_names, cutoff=cutoff)
    try:
        closest = matches[0]
        return tracks_by_name[closest]
    except IndexError:
        return get_tracks(title, cutoff=0.3)
    return None
    
def pick_track(tracks, track_listened, id_series):
    same_artists = [t for t in tracks if t.artist == track_listened.artist]
    if len(same_artists) == 0:
        same_artists = [t for t in tracks if t.artist == 'nan']
        if len(same_artists) == 0:
            return None
    if len(same_artists) == 1:
        return same_artists[0]
    by_album = {t.album: t for t in same_artists}
    try:
        previous_track = df_t.iloc[id_series[len(id_series) - 1]]
        if previous_track.artist == track_listened.artist:
            return by_album[previous_track.album]
    except (KeyError, TypeError):
        pass
    return same_artists[0]
    
id_series = deque()
not_matched = []
NotMatched = namedtuple('NotMatched', ['track', 'candidates'])

for track_listened in tqdm(list(df.itertuples(index=True))):
    tracks = get_tracks(track_listened.title)
    if tracks is None:
        id_series.append(None)
        not_matched.append(NotMatched(track_listened))
        continue
    track = pick_track(tracks, track_listened, id_series)
    if track is None:
        id_series.append(None)
        not_matched.append(NotMatched(track_listened, tracks))
        continue
    id_series.append(track.Index)

100%|██████████| 12412/12412 [00:09<00:00, 1315.80it/s]


In [9]:
df_nm = pd.DataFrame([t.track for t in not_matched])
# with pd.option_context('display.max_rows', 500):
    # display(df_nm)
display(df_nm)

,Index,artist,title,time
0,4610,Leaves' Eyes,To France,2020-01-03T10:21:20.687Z
1,4611,Leaves' Eyes,Krakevisa,2020-01-03T10:16:43.386Z
2,4612,Leaves' Eyes,Velvet Heart,2020-01-03T10:12:08.894Z
3,4613,Leaves' Eyes,Étaín,2020-01-03T10:08:26.770Z
4,4614,Leaves' Eyes,Spirits' Masquerade,2020-01-03T10:04:28.614Z
...,...,...,...,...
74,12049,Two Steps from Hell,Battleborne - Instrumental,2019-05-17T12:59:58.281Z
75,12252,Two Steps from Hell,Star Sky - Instrumental,2019-05-14T11:58:44.891Z
76,12253,Two Steps from Hell,Battleborne - Instrumental,2019-05-14T11:58:23.258Z
77,12358,Two Steps from Hell,Battleborne - Instrumental,2019-05-12T16:20:19.579Z


In [10]:
df['song_id'] = id_series

# To DB

In [11]:
from db import DBConn
from sqlalchemy.ext.declarative import declarative_base

DBConn()
Base = declarative_base()

In [12]:
import sqlalchemy as sa
    
class AlbumSong(Base):
    __tablename__ = 'AlbumSong'
    __table_args__ = {'schema': SCHEMA}
    
    id = sa.Column(
        sa.BigInteger(),
        primary_key=True,
        nullable=False,
        unique=True,
        autoincrement=True,
    )
    
    title = sa.Column(sa.String(256), nullable=False)
    album = sa.Column(sa.String(256), nullable=False)
    artist = sa.Column(sa.String(256), nullable=False)
    duration = sa.Column(sa.String(256), nullable=False)
    play_count = sa.Column(sa.String(256), nullable=False)
    
class MusicListened(Base):
    __tablename__ = 'MusicListened'
    __table_args__ = {'schema': SCHEMA}
    
    id = sa.Column(
        sa.BigInteger(),
        primary_key=True,
        nullable=False,
        unique=True,
        autoincrement=True,
    )
    
    artist = sa.Column(sa.String(256), nullable=False)
    title = sa.Column(sa.Text(), nullable=False)
    time = sa.Column(sa.DateTime(), nullable=False)
    
    song_id = sa.Column(sa.BigInteger(), sa.ForeignKey(f'{SCHEMA}.AlbumSong.id'), nullable=True)


In [13]:
DBConn.engine.execute(f'DROP SCHEMA IF EXISTS {SCHEMA} CASCADE')
DBConn.engine.execute(f'CREATE SCHEMA IF NOT EXISTS {SCHEMA}')
Base.metadata.create_all(DBConn.engine)

In [14]:
with DBConn.get_session() as db:
    for track in df_t.itertuples(index=True):
        track = track._asdict()
        track['id'] = track['Index']
        del track['Index']
        
        song = AlbumSong(**track)
        db.add(song)
    db.commit()
    
    for track in df.itertuples(index=False):
        track = track._asdict()
        
        ml = MusicListened(**track)
        db.add(ml)
    
    db.commit()